#### Author: Anilkumar Vempadapu
Problem statment - There are times when a user writes Good, Nice App or any other positive text, in the review and gives 1-star rating. Your goal is to identify the reviews where the semantics of review text does not match rating. 

Goal is to identify such ratings where review text is good, but rating is negative.

In [1]:
import pandas as pd

In [2]:
reviews = pd.read_csv('..\\data\\chrome_reviews.csv').set_index('ID')
reviews = reviews[reviews["Text"].notna()]

In [3]:
reviews_text = reviews[["Text"]]
reviews_rating = reviews[["Star"]]

In this approach, we use a pre-trained model Flair to carry the sentiment analysis, we import the package from Flair and then whenever new data is uploaded we predict the positivity in each senetnce and compare with the rating provided by user and further steps are carried accordingly.

In [4]:
import flair
from flair.models import TextClassifier
flair_sentiment = TextClassifier.load('en-sentiment')

2022-07-02 12:26:58,672 loading file C:\Users\EHSIICV\.flair\models\sentiment-en-mix-distillbert_4.pt


In [5]:
def predict_rating(text):
    sentence=flair.data.Sentence(text)
    flair_sentiment.predict(sentence)
    total_sentiment = sentence.labels
    
    if "NEGATIVE" in str(total_sentiment[0]):
        if sentence.score > 0.5:
            return 1
        else:
            return 2
    else:
        if sentence.score > 0.75:
            return 5
        elif sentence.score > 0.3:
            return 4
        else:
            return 3

In [6]:
reviews_text['Rating'] = reviews_text['Text'].apply(lambda x: predict_rating(x))

<ipython-input-6-514d1a337d3b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_text['Rating'] = reviews_text['Text'].apply(lambda x: predict_rating(x))


In [7]:
final_review = pd.concat([reviews_text, reviews_rating],axis=1)

In [8]:
reviews_of_interest = final_review[final_review['Rating']-final_review['Star']>2]

##### Flair model is used to predict the sentiment given a text, based on the predicted sentiment, a rating is dervied and if there exists a difference of more than 2 rating points from predicted to given rating(i.e. star) then the review is stored and provided for further analysis.

The model is stored in a pickle file format for deploying and making live inferences.

In [9]:
reviews_of_interest.to_csv("..\\data\\Reviews of interest.csv")

In [10]:
import pickle
pickle.dump(flair_sentiment, open('..\\models\\flair_sa.pkl', 'wb'))

In [11]:
import pickle
with open('..\\models\\flair_sa.pkl', 'rb') as file:
       sentence_model = pickle.load(file)